In [1]:
import pandas as pd
import numpy as np
import sklearn as sklearn 
import datetime
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
import math 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
# Import dataset
df = pd.read_csv(r'C:\Users\Yuva S Korlaband\Documents\CCA Model Weekly\carbon_subset_Weekly_df.csv')

# Drop unnecessary columns
cols_to_drop = [
    'index',  'Benchmark', 'Front_Weekly_min', 'Front_Weekly_max',
    'CA_Mean', 'CA_Median', 'CA_Percent Above Floor', 'CA_Median allowance price',
    'AA_Maximum', 'CA_Clearing Price', 'AA_Mean', 'AA_Median', 'CA_Maximum',
    'CA_Cleared Above Floor', 'AA_Median allowance price', 'AA_Floor Price',
    'AA_Clearing Price', 'CA_Vintage', 'AA_Percent above Floor',
    'AA_Cleared Above Floor', 'CA_Minimum', 'CA_Floor Price', 'AA_Vintage'
]
df.drop(cols_to_drop, axis=1, inplace=True)

# Define target (Y) and feature column (X) names
target_col = ['Front', 'date']
feature_cols = df.columns.difference(target_col).tolist()

df_target = df[target_col]
df_features = df[feature_cols]

# DateTime encoding
df_target['date'] = pd.to_datetime(df_target['date']).dt.strftime('%Y-%m-%d').astype(str)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 69 columns):
 #   Column                                                                                         Non-Null Count  Dtype  
---  ------                                                                                         --------------  -----  
 0   date                                                                                           375 non-null    object 
 1   8 Open_Interest_All                                                                            375 non-null    float64
 2   9 Prod_Merc_Positions_Long_All                                                                 375 non-null    float64
 3   10 Prod_Merc_Positions_Short_All                                                               375 non-null    float64
 4   11 Swap_Positions_Long_All                                                                     375 non-null    float64
 5   12 Swap__Positions_Sho

In [4]:
new_df = df[["date", "Front"]]

In [5]:
new_df

,date,Front
0,2015-06-02,12.530000
1,2015-06-09,12.505714
2,2015-06-16,12.540000
3,2015-06-23,12.570000
4,2015-06-30,12.607143
...,...,...
370,2022-07-05,30.192857
371,2022-07-12,29.578571
372,2022-07-19,27.997143
373,2022-07-26,27.715714


In [15]:
# Split the DataFrame into two parts: one with "date" column and one without
date_column = df["date"]
other_columns = df.drop("date", axis=1)

# Shift the part without the "date" column by 2 rows
shifted_other_columns = other_columns.shift(periods=-2)

# Concatenate the "date" column and the shifted part back together
shifted_df = pd.concat([date_column, shifted_other_columns], axis=1)
shifted_df.rename(columns={'Front': 'Front_2'}, inplace=True)
# Display the shifted DataFrame
print(shifted_df)


           date  8 Open_Interest_All  9 Prod_Merc_Positions_Long_All  \
0    2015-06-02              24900.0                          8856.0   
1    2015-06-09              25980.0                          8790.0   
2    2015-06-16              20940.0                          8508.0   
3    2015-06-23              21766.0                          9297.0   
4    2015-06-30              22679.0                          9515.0   
..          ...                  ...                             ...   
370  2022-07-05             165241.0                         32478.0   
371  2022-07-12             169353.0                         34419.0   
372  2022-07-19             155861.0                         30019.0   
373  2022-07-26                  NaN                             NaN   
374  2022-08-02                  NaN                             NaN   

     10 Prod_Merc_Positions_Short_All  11 Swap_Positions_Long_All  \
0                              6134.0                         0.0 

In [14]:
df

,date,8 Open_Interest_All,9 Prod_Merc_Positions_Long_All,10 Prod_Merc_Positions_Short_All,11 Swap_Positions_Long_All,12 Swap__Positions_Short_All,14 M_Money_Positions_Long_All,15 M_Money_Positions_Short_All,56 56 Change_in_Open_Interest_All,57 57 Change_in_Prod_Merc_Long_All,...,1 Market_and_Exchange_Names_CALIF CARBON 22 - ICE FUTURES ENERGY DIV,1 Market_and_Exchange_Names_CALIF CARBON VINTAGE 2020 - ICE FUTURES ENERGY DIV,1 Market_and_Exchange_Names_CALIFORNIA CARBON ALLOWANCE VINTAGE 2015 - ICE FUTURES ENERGY DIV,CALIF CARBON VINTAGE 2020 - ICE FUTURES ENERGY DIV,CALIF CARBON 22 - ICE FUTURES ENERGY DIV,CALIF CARBON ALLOWANCE V2022 - ICE FUTURES ENERGY DIV,1 Market_and_Exchange_Names_CALIF CARBON 23 - ICE FUTURES ENERGY DIV,CALIF CARBON 23 - ICE FUTURES ENERGY DIV,CALIF CARBON CURRENT AUCTION - ICE FUTURES ENERGY DIV,week_idx
0,2015-06-02,19591.0,8658.0,4945.0,0.0,8109.0,0.0,0.0,-3407.0,226.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,23
1,2015-06-09,21881.0,8795.0,6020.0,0.0,8185.0,0.0,0.0,2290.0,137.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,24
2,2015-06-16,24900.0,8856.0,6134.0,0.0,8335.0,0.0,0.0,3019.0,61.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,25
3,2015-06-23,25980.0,8790.0,6189.0,0.0,8749.0,0.0,0.0,1080.0,-66.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,26
4,2015-06-30,20940.0,8508.0,5501.0,0.0,7886.0,0.0,0.0,-5040.0,-282.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,2022-07-05,166829.0,32064.0,31274.0,5636.0,53227.0,79281.0,10255.0,-3016.0,-1054.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27
371,2022-07-12,164625.0,31296.0,30368.0,5721.0,51627.0,76768.0,10430.0,-2204.0,-768.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28
372,2022-07-19,165241.0,32478.0,29553.0,5462.0,47061.0,70445.0,10704.0,616.0,1182.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29
373,2022-07-26,169353.0,34419.0,29515.0,5537.0,46083.0,66940.0,11068.0,4112.0,1941.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30


In [11]:
df

,date,8 Open_Interest_All,9 Prod_Merc_Positions_Long_All,10 Prod_Merc_Positions_Short_All,11 Swap_Positions_Long_All,12 Swap__Positions_Short_All,14 M_Money_Positions_Long_All,15 M_Money_Positions_Short_All,56 56 Change_in_Open_Interest_All,57 57 Change_in_Prod_Merc_Long_All,...,1 Market_and_Exchange_Names_CALIF CARBON 22 - ICE FUTURES ENERGY DIV,1 Market_and_Exchange_Names_CALIF CARBON VINTAGE 2020 - ICE FUTURES ENERGY DIV,1 Market_and_Exchange_Names_CALIFORNIA CARBON ALLOWANCE VINTAGE 2015 - ICE FUTURES ENERGY DIV,CALIF CARBON VINTAGE 2020 - ICE FUTURES ENERGY DIV,CALIF CARBON 22 - ICE FUTURES ENERGY DIV,CALIF CARBON ALLOWANCE V2022 - ICE FUTURES ENERGY DIV,1 Market_and_Exchange_Names_CALIF CARBON 23 - ICE FUTURES ENERGY DIV,CALIF CARBON 23 - ICE FUTURES ENERGY DIV,CALIF CARBON CURRENT AUCTION - ICE FUTURES ENERGY DIV,week_idx
0,2015-06-02,19591.0,8658.0,4945.0,0.0,8109.0,0.0,0.0,-3407.0,226.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,23
1,2015-06-09,21881.0,8795.0,6020.0,0.0,8185.0,0.0,0.0,2290.0,137.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,24
2,2015-06-16,24900.0,8856.0,6134.0,0.0,8335.0,0.0,0.0,3019.0,61.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,25
3,2015-06-23,25980.0,8790.0,6189.0,0.0,8749.0,0.0,0.0,1080.0,-66.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,26
4,2015-06-30,20940.0,8508.0,5501.0,0.0,7886.0,0.0,0.0,-5040.0,-282.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,2022-07-05,166829.0,32064.0,31274.0,5636.0,53227.0,79281.0,10255.0,-3016.0,-1054.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27
371,2022-07-12,164625.0,31296.0,30368.0,5721.0,51627.0,76768.0,10430.0,-2204.0,-768.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28
372,2022-07-19,165241.0,32478.0,29553.0,5462.0,47061.0,70445.0,10704.0,616.0,1182.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29
373,2022-07-26,169353.0,34419.0,29515.0,5537.0,46083.0,66940.0,11068.0,4112.0,1941.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30


In [16]:
result_df = new_df.merge(shifted_df, on="date")

In [21]:
result_df

,date,Front,8 Open_Interest_All,9 Prod_Merc_Positions_Long_All,10 Prod_Merc_Positions_Short_All,11 Swap_Positions_Long_All,12 Swap__Positions_Short_All,14 M_Money_Positions_Long_All,15 M_Money_Positions_Short_All,56 56 Change_in_Open_Interest_All,...,CALIF CARBON ALLOWANCE V2022 - ICE FUTURES ENERGY DIV,1 Market_and_Exchange_Names_CALIF CARBON 23 - ICE FUTURES ENERGY DIV,CALIF CARBON 23 - ICE FUTURES ENERGY DIV,CALIF CARBON CURRENT AUCTION - ICE FUTURES ENERGY DIV,week_idx,quarter,year,month,weekday,auction_idx
0,2015-06-02,12.530000,24900.0,8856.0,6134.0,0.0,8335.0,0.0,0.0,3019.0,...,0.0,0.0,0.0,0.0,25.0,2,2015,6,1,0
1,2015-06-09,12.505714,25980.0,8790.0,6189.0,0.0,8749.0,0.0,0.0,1080.0,...,0.0,0.0,0.0,0.0,26.0,2,2015,6,1,0
2,2015-06-16,12.540000,20940.0,8508.0,5501.0,0.0,7886.0,0.0,0.0,-5040.0,...,0.0,0.0,0.0,0.0,27.0,2,2015,6,1,0
3,2015-06-23,12.570000,21766.0,9297.0,5601.0,0.0,8086.0,0.0,0.0,826.0,...,0.0,0.0,0.0,0.0,28.0,2,2015,6,1,0
4,2015-06-30,12.607143,22679.0,9515.0,6201.0,0.0,8229.0,0.0,0.0,913.0,...,0.0,0.0,0.0,0.0,29.0,2,2015,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,2022-07-05,30.192857,165241.0,32478.0,29553.0,5462.0,47061.0,70445.0,10704.0,616.0,...,0.0,0.0,0.0,0.0,29.0,3,2022,7,1,0
371,2022-07-12,29.578571,169353.0,34419.0,29515.0,5537.0,46083.0,66940.0,11068.0,4112.0,...,0.0,0.0,0.0,0.0,30.0,3,2022,7,1,0
372,2022-07-19,27.997143,155861.0,30019.0,28669.0,8233.0,45055.0,66065.0,11057.0,-13492.0,...,0.0,0.0,0.0,0.0,31.0,3,2022,7,1,0
373,2022-07-26,27.715714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3,2022,7,1,0


In [19]:
result_df['date'] = pd.to_datetime(result_df['date'])
result_df['quarter'] = result_df['date'].dt.quarter
result_df['year'] = result_df['date'].dt.year
result_df['month'] = result_df['date'].dt.month
#result_df['week'] = result_df['date'].dt.week
result_df['weekday'] = result_df['date'].dt.weekday

In [20]:
auct_months = [2, 5, 8, 11]
result_df['auction_idx'] = np.where(result_df['month'].isin(auct_months), 1, 0)

In [35]:
result_df = result_df.dropna()

In [36]:
remove_feature_cols = (['date', 'Front'])

feature_cols = [i for i in list(df.columns) if i not in remove_feature_cols]

In [37]:
def split_by_idx(row_limit):
    test_set_idx = list(range(4,100,10))
    X_train = df[feature_cols][~df.index.isin(test_set_idx)][:row_limit] 
    X_test = df[feature_cols][df.index.isin(test_set_idx)][:row_limit]
    y_train = df[target_col[0]][~df.index.isin(test_set_idx)][:row_limit]
    y_test = df[target_col[0]][df.index.isin(test_set_idx)][:row_limit]
    return X_train, y_train, X_test, y_test


# prediction horizon: 1 month
X_train1, y_train1, X_test1, y_test1 = split_by_idx(df[feature_cols].shape[0]-2)

In [38]:
def select_features(X_train, y_train, X_test, N_KBest):
    if N_KBest is None:
        fs = SelectKBest(score_func=f_regression, k='all')
        #fs = SelectKBest(score_func=mutual_info_regression, k='all')
    else:
        fs = SelectKBest(score_func=f_regression, k=N_KBest)
        #fs = SelectKBest(score_func=mutual_info_regression, k=N_KBest)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs
 

X_train_fs1, X_test1_fs, fs1 = select_features(X_train1, y_train1, X_test1, None)

In [39]:
def get_KBest(X_train, fs):
# what are scores for the features
    KBest_cols = []
    for i in range(len(fs.scores_)):
        if fs.scores_[i] >= 5:
        #print(X_train1.columns[i]+':', fs.scores_[i])
            KBest_cols.append(X_train.columns[i]) 
            
    N_KBest = len(KBest_cols)
    return KBest_cols, N_KBest


KBest_cols1, N_KBest1 = get_KBest(X_train1, fs1)

In [40]:
X_train_fs1, X_test1_fs, fs1 = select_features(X_train1, y_train1, X_test1, N_KBest1)

In [41]:

def scale_inputs(X_train, X_test, X_pred):
    scaler = StandardScaler().fit(X_train)
    train_sc = scaler.transform(X_train)
    test_sc = scaler.transform(X_test)
    pred_sc = scaler.transform(X_pred)
    
    return train_sc, test_sc, pred_sc


n_rows1 = df[KBest_cols1].shape[0]
X_pred1 = df[KBest_cols1][(n_rows1-2):n_rows1]

In [42]:
def get_best_params(X_train, y_train):

    clf_rfg = RandomForestRegressor(random_state=42)
    
    param_grid = { 
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth' : [2,6,8],
        "min_samples_split" : [2,4,8],
        'bootstrap': [True, False]}
    
    CV_rfc = GridSearchCV(estimator=clf_rfg, param_grid=param_grid, cv= 5)
    CV_rfc.fit(X_train, y_train)
    
    return CV_rfc.best_params_


#best_params1 = get_best_params(X_train1, y_train1)
best_params1_rfg = get_best_params(X_train_fs1, y_train1)

In [43]:
def get_best_params(X_train, y_train):

    clf_dtr = DecisionTreeRegressor(random_state=42)
    
    param_grid = { 
           "splitter":["best","random"],
           "max_depth" : [1, 5, 9, 12],
           "min_samples_leaf":[1, 3, 5, 8, 10],
           "min_weight_fraction_leaf":[0.1, 0.3, 0.5, 0.7, 0.9],
           "max_features":["auto", "log2", "sqrt", None],
           "max_leaf_nodes":[None, 10, 30, 50, 70, 90] }
    
    CV_rfc = GridSearchCV(estimator=clf_dtr, param_grid=param_grid, cv= 5)
    CV_rfc.fit(X_train, y_train)
    
    return CV_rfc.best_params_


#best_params1 = get_best_params(X_train1, y_train1)
best_params1_dtr = get_best_params(X_train_fs1, y_train1)

In [44]:
def get_best_params(X_train, y_train):

    clf_abr = AdaBoostRegressor(random_state=42)
    
    param_grid = {
        'n_estimators':[500,1000,2000],
        'learning_rate':[.001,0.01,.1]}
    
    CV_abr = GridSearchCV(estimator=clf_abr, param_grid=param_grid, cv= 5)
    CV_abr.fit(X_train, y_train)
    
    return CV_abr.best_params_


#best_params1 = get_best_params(X_train1, y_train1)
best_params1_abr = get_best_params(X_train_fs1, y_train1)

In [45]:
def train_models(X_train, y_train, best_params_rfg, best_params_dtr, best_params_abr):
    
    clf_rfg = RandomForestRegressor(random_state=42,
                                    max_features = best_params_rfg['max_features'],
                                    bootstrap = best_params_rfg['bootstrap'],
                                    max_depth = best_params_rfg['max_depth'],
                                    min_samples_split = best_params_rfg['min_samples_split'])
    clf_rfg.fit(X_train, y_train)
    
    
    clf_dtr = DecisionTreeRegressor(random_state=42, 
                                    splitter = best_params_dtr['splitter'],
                                    max_depth = best_params_dtr["max_depth"],
                                    min_samples_leaf = best_params_dtr["min_samples_leaf"],
                                    min_weight_fraction_leaf = best_params_dtr["min_weight_fraction_leaf"],
                                    max_features = best_params_dtr["max_features"],
                                    max_leaf_nodes = best_params_dtr["max_leaf_nodes"])
    clf_dtr.fit(X_train, y_train)

    
    clf_abr = AdaBoostRegressor(random_state = 42,
                                        n_estimators = best_params_abr['n_estimators'],
                                        learning_rate = best_params_abr['learning_rate'])
    clf_abr.fit(X_train, y_train)
    
    return clf_rfg, clf_dtr, clf_abr



def get_predictions(models, df):
    
    rfg = models[0].predict(df)
    dtr = models[1].predict(df)
    abr = models[2].predict(df)
    
    result = np.mean([rfg, dtr, abr], axis=0)
    #print(result)
    return result


def combine_predictions(predictionsA, predictionsB):
    lenA = len(list(predictionsA))
    lenB = len(list(predictionsB))
    return list(predictionsA) + list(predictionsB[lenA:lenB])


# prediction horizon: 1 month
models1 = train_models(X_train_fs1, y_train1, best_params1_rfg, best_params1_dtr, best_params1_abr)
predictions1 = get_predictions(models1, X_pred1)

In [46]:
def get_performance_metrics(y_test, y_pred):
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('R-squared:', metrics.r2_score(y_test, y_pred))
    

print('Model metrics: predictions 1 month')    
get_performance_metrics(y_test1, get_predictions(models1, X_test1_fs))
print('\n')

Model metrics: predictions 1 month
Mean Absolute Error: 0.28192410451937755
Mean Squared Error: 0.09642192431382123
Root Mean Squared Error: 0.3105187986480387
R-squared: -0.13535446560942188


